In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import Versions, Version, VersionGroups, RenameType
from depsurf.output import save_df

import re


def get_suffix(x):
    suffix = x.split(".", 1)[-1]
    return re.sub(r"\.\d+", "", suffix)


results = {}


def count_rename(v: Version):
    all_names = {sym["name"]: sym["bind"] for sym in v.img.symtab.iter_funcs()}
    result = {t: 0 for t in RenameType}

    for func in all_names.keys():
        if "." not in func:
            continue
        suffix = get_suffix(func)
        if suffix == "localalias":
            continue
        result[RenameType.from_suffix(suffix)] += 1

    return {
        **result,
        "Total": len(all_names),
        "Renamed": sum(result.values()),
        "GCC": v.img.gcc_version
    }

df = VersionGroups(Versions.REGULAR, Versions.ARCH, Versions.FLAVOR).apply(count_rename)
save_df(df, "rename")

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.8.0-22-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.10.0-19-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.13.0-16-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.15.0-20-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.18.0-10-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/5.0.0-13-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Dow

ISRA Const Propagation Part  Cold   ≥2  Total Renamed     GCC
Regular 4.4.0-21-generic-amd64     575               196  522     0   64  38401    1357   5.3.1
        4.8.0-22-generic-amd64     577               233  561     0   63  40274    1434   6.2.0
        4.10.0-19-generic-amd64    588               249  575     0   59  41252    1471   6.3.0
        4.13.0-16-generic-amd64    605               249  724     0   69  43021    1647   7.2.0
        4.15.0-20-generic-amd64    626               257  759     0   65  44662    1707   7.3.0
        4.18.0-10-generic-amd64    706               262  735  1875  161  48438    3739   8.2.0
        5.0.0-13-generic-amd64     717               261  749  1910  157  49281    3794   8.3.0
        5.3.0-18-generic-amd64     782               276  766  4348  330  53711    6502   9.2.1
        5.4.0-26-generic-amd64     780               275  768  3223  244  53486    5290   9.3.0
        5.8.0-25-generic-amd64     602               640  970  3621  369  58346    6202  10.2.0
        5.11.0-16-generic-amd64    623               661  996  3827  393  61863    6500  10.3.0
        5.13.0-19-generic-amd64    516               598  723  3830  312  60245    5979  11.2.0
        5.15.0-25-generic-amd64    510               613  707  8064  548  65484   10442  11.2.0
        5.19.0-21-generic-amd64    547               673  519  8513  506  67567   10758  12.2.0
        6.2.0-20-generic-amd64     562               735  494  5718  420  66993    7929  12.2.0
        6.5.0-9-generic-amd64      813               567  500  6003  420  69967    8303  13.2.0
        6.8.0-22-generic-amd64     850               583  530  6557  435  74028    8955  13.2.0
Arch    5.4.0-26-generic-arm64     967               260  785     0   98  51766    2110   9.3.0
        5.4.0-26-generic-armhf       0               447  804     0    4  50880    1255   9.3.0
        5.4.0-26-generic-ppc64el   589               198  671     0   66  45017    1524   9.3.0
        5.4.0-26-generic-s390x     667               166  532     0   75  35219    1440   9.3.0
Flavor  5.4.0-26-lowlatency-amd64  780               277  774  3226  243  53514    5300   9.3.0
        5.4.0-1009-aws-amd64       759               265  745  2995  234  51699    4998   9.3.0
        5.4.0-1009-gcp-amd64       786               279  766  3215  245  53618    5291   9.3.0
        5.4.0-1009-oracle-amd64    808               295  785  3397  257  55014    5542   9.3.0
        5.4.0-1010-azure-amd64     727               262  709  2819  226  50331    4743   9.3.0

In [1]:
from depsurf.output import plot_yticks, save_fig, bold

import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8, 3))

colors = {
    "isra": "C0",
    "part": "C1",
    "cold": "C2",
    "constprop": "C3",
    "≥2": "C4",
}

xs = np.arange(len(df.index))
bottom = np.zeros(len(df.index))
for col in list(colors):
    ax.bar(xs, df[col], bottom=bottom, label=col, color=colors[col])
    bottom += df[col]

for x, b, p in zip(xs, bottom, df["Renamed"] / df["Total"]):
    ax.text(x, b, f"{p:.0%}", ha="center", va="bottom", fontsize=8)

labels = [bold(v.short_version) if v.lts else v.short_version for v in df.index.get_level_values(1)]
labels = [f"{l}\n{gcc[:gcc.rfind('.')]}" for l, gcc in zip(labels, df["GCC"])]
ax.set_xticks(xs, labels, rotation=0)

plot_yticks(ax)

ax.set_ylabel("Number of renamed functions")
ax.set_xlabel("Linux kernel version and GCC version")
ax.legend(loc="upper left")

ax.tick_params(labelsize=9)

save_fig(fig, "rename", close=False)

4.15 -> 4.18:
- `-freorder-blocks-and-partition` enabed by default in GCC 8

- https://gcc.gnu.org/gcc-8/changes.html

5.0 -> 5.3:
- "Hot/cold partitioning is now more precise and aggressive." since GCC 9

- https://gcc.gnu.org/gcc-9/changes.html

5.13 -> 5.15:

- Non-static functions renamed e.g., `ipv6_local_rxpmtu`

5.19 -> 6.2: 

- `-falign-functions=16` added since Linux 6.2

- https://github.com/torvalds/linux/commit/c27cd083cfb9d392f304657ed00fcde1136704e7

- https://github.com/torvalds/linux/commit/d49a0626216b95cd4bf696f6acf55f39a16ab0bb

In [ ]:
def get_flags(version):
    return set(filter_version(version=version)[0].img.dwarf.flags)


get_flags("5.15") - get_flags("5.13")

{'-fsanitize=bool',
 '-fsanitize=bounds',
 '-fsanitize=enum',
 '-fsanitize=shift',
 '-gdwarf-5'}

In [ ]:
def get_cold_funcs(version):
    df = filter_version(version=version)[0].img.symtab.funcs_renamed
    # return df
    names = df["name"]
    return set(names[names.str.contains(".cold")])


# get_cold_funcs("5.15") - get_cold_funcs("5.13")

# df = get_cold_funcs("5.15")

# df